In [1]:
from transformers import BertForQuestionAnswering, AutoTokenizer
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [2]:
from transformers import pipeline
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [3]:
a1_path = "./Project-Data/set4/a1.txt"
a2_path = "./Project-Data/set4/a2.txt"
a3_path = "./Project-Data/set4/a3.txt"
a4_path = "./Project-Data/set4/a4.txt"
a5_path = "./Project-Data/set4/a5.txt"
a6_path = "./Project-Data/set4/a6.txt"
a7_path = "./Project-Data/set4/a7.txt"
a8_path = "./Project-Data/set4/a8.txt"
a9_path = "./Project-Data/set4/a9.txt"

In [4]:
import io

In [5]:
from nltk.corpus import stopwords

In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
def get_article(path):
    with io.open(path, "r", encoding="utf-8") as f:
        article = f.read()
    return article

In [8]:
a1 = get_article(a1_path)
a2 = get_article(a2_path)
a3 = get_article(a3_path)
a4 = get_article(a4_path)
a5 = get_article(a5_path)
a6 = get_article(a6_path)
a7 = get_article(a7_path)
a8 = get_article(a8_path)
a9 = get_article(a9_path)

In [9]:
articles = [a1, a2, a3, a4, a5, a6, a7, a8, a9]

In [ ]:
# topic recognition 
def movie(text):
    # assume text given is only the first sentence
    return re.search("film", text, re.IGNORECASE) != None

def football(text):
    return re.searh("soccer player|footballer", text, re.IGNORECASE) != None

def constellation(text):
    return re.search("constellation", text, re.IGNORECASE) != None

def language(text):
    return re.search("language", text, re.IGNORECASE) != None

In [10]:
film_questions = [
    "What is the genre of the film?",
    "When was the film released?", 
    "Who is the director of the film?",
    "What is the box office of the film?",
    "What is the film criticised for?",
    "What are some characters in the film?",
    "Did the film win any awards?",
    "Is the film a success?"
]

In [11]:
def handle_1st(context, question):
    # find in 1st sentence
    first_sent = context.split("\n\n\n")[1].split(".")[0]
    ans = nlp({
        "question": question,
        "context": first_sent
    })
    return ans["answer"]

In [12]:
def handle_intro(context, question):
    intro = context.split("\n\n\n")[1]
    ans = nlp({
        "question": question,
        "context": intro
    })
    return ans["answer"]

In [25]:
film_keywords_1st_sent = set(["genre", "country", "time"])
film_1st_sent_handler = (film_keywords_1st_sent, handle_1st)
film_keywords_intro = set(["written", "directed", "edited", "produced", "box office", \
                            "review", "acclaimed", "praised", "criticized", "nominate", "award", "success"])
film_intro_handler = (film_keywords_intro, handle_intro)

In [14]:
film_handlers = [film_1st_sent_handler, film_intro_handler]

In [23]:
def hardcode(context, question, handlers):
    for keywords, handler in handlers:
        for keyword in keywords:
            if keyword in question:
                ans = handler(context, question)
                return ans
    return None # answer can't be found through hardcode

In [26]:
def check1(question)
for question in film_questions:
    print(question)
    ans = hardcode(a1, question, film_handlers)
    print(ans)
    

What is the genre of the film?
romantic comedy-drama
When was the film released?
None
Who is the director of the film?
Michel Hazanavicius
What is the box office of the film?
won five
What is the film criticised for?
None
What are some characters in the film?
None
Did the film win any awards?
the most of any film from 2011
Is the film a success?
received highly positive reviews from critics and won many accolades.


In [ ]:
answers = []
def check1(context, question):
    ans = None
    if movie(context):
        ans = hardcode(context, question, film_handlers)
    elif football(context):
        ans = hardcode(context, question, football_handlers)
    elif constellation(context):
        ans = hardcode(context, question, constellation_handlers)
    elif language(context):
        ans = hardcode(context, question, language_handlers)
    
    if ans:
        answers.append(ans)
    check2(context, question)